# Actions

> Definiting actions, how to register them, etc.

In [ ]:
#| default_exp actions

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from triggerkit import core
from typing import Dict, List, Callable, Optional, Any, Union

In [ ]:
#| export

core.ACTION_REGISTRY: Dict[str, Callable] = {}

def register(name: str, description: Optional[str] = None):
    """
    Register a function as an action in the global registry.
    
    Args:
        name: Unique identifier for the action
        description: Optional description of what the action does
        
    Returns:
        Decorator function that registers the decorated function
        
    Raises:
        ValueError: If an action with the same name already exists
    """
    def decorator(fn: Callable) -> Callable:
        if name in core.ACTION_REGISTRY and core.ACTION_REGISTRY[name] != fn:
            raise ValueError(f"An action named '{name}' is already registered.")
        
        # Add metadata to the function
        fn._action_name = name
        fn._action_description = description
        
        # Add to registry
        core.ACTION_REGISTRY[name] = fn
        
        # Preserve function metadata like docstrings
        @functools.wraps(fn)
        def wrapper(*args, **kwargs):
            return fn(*args, **kwargs)
        
        return wrapper
    
    return decorator

In [ ]:
#| export

def list_available():
    """List all registered actions with their descriptions."""
    print("Available actions:")
    for name, fn in core.ACTION_REGISTRY.items():
        description = getattr(fn, "_action_description", "No description available")
        print(f"  - {name}: {description}")
    return core.ACTION_REGISTRY

def get_info(name: str) -> Dict[str, Any]:
    """Get detailed information about a specific action."""
    if name not in core.ACTION_REGISTRY:
        raise ValueError(f"No action named '{name}' is registered.")
    
    fn = core.ACTION_REGISTRY[name]
    info = {
        "name": name,
        "description": getattr(fn, "_action_description", "No description available"),
        "docstring": fn.__doc__ or "No documentation",
        "signature": str(inspect.signature(fn))
    }
    return info

In [ ]:
#| export

def run(action_name: str, data: List[Dict[str, Any]]) -> Any:
    """
    Run a registered action with the provided data.
    
    Args:
        action_name: Name of the registered action
        data: Data to pass to the action
        
    Returns:
        Result of the action
    """
    if action_name not in core.ACTION_REGISTRY:
        raise ValueError(f"No action named '{action_name}' is registered")
    
    action = core.ACTION_REGISTRY[action_name]
    try:
        result = action(data)
        core.logger.info(f"Successfully executed action '{action_name}'")
        return result
    except Exception as e:
        core.logger.error(f"Failed to execute action '{action_name}': {str(e)}")
        raise

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()